# Test of our results

In this file, we demontsrate how well our model performs compared to the baseline and we find the best parameters for our preprocessing algorithm.

In [2]:
import pandas as pd
from pipeline import get_full_pipeline_with_model
from baseline_pipeline import get_baseline_full_pipeline
from sklearn.model_selection import train_test_split

data = pd.read_csv('CarsData.csv')

full_pipeline_with_model = get_full_pipeline_with_model(data, 'price')
baseline_full_pipeline = get_baseline_full_pipeline(data, 'price')

y = data.price
X = data.drop('price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Here we search for the best parameters:

In [1]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")


grid_params = [
    {
        'full_pipeline__select_from_model__threshold' : [50, 100, 200],
        'full_pipeline__feature_extraction__method': ['PCA'],
        'full_pipeline__feature_extraction__n_components': [0.9, 0.95, 0.99]
    },
    {
        'full_pipeline__select_from_model__threshold' : [50, 100, 200],
        'full_pipeline__feature_extraction__method': ['LDA'],
        'full_pipeline__feature_extraction__n_components': [15, 30, 50]
    }
]

grid = GridSearchCV(full_pipeline_with_model, grid_params, verbose=3, cv = 3, scoring = 'neg_mean_squared_error')
grid.fit(X_train, y_train)

NameError: name 'full_pipeline_with_model' is not defined

In [6]:
grid.best_params_

{'feature_extraction__method': 'LDA',
 'feature_extraction__n_components': 30,
 'select_from_model__threshold': 200}

In [7]:
model = grid.best_estimator_
model

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Variance_threshold',
                                                                   VarianceThreshold(threshold=0.01)),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage', 'tax',
                                                   'mpg', 'engineSize']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('Variance_threshold',
                                                                   VarianceThreshold(threshold=0.01))]),
                                                  [...
                ('dense',
                 FunctionTransformer(accept_sparse=True,
                                     func=<function <lambda> at 0x7fe0bf404040>)),
                ('select_from_model',
                 SelectFromModel(estimator=LassoCV(), threshold=200)),
                ('feature_extraction',
                 FeatureExtraction(method='LDA', n_components=30)),
                ('model',
                 MLPRegressor(batch_size=8, hidden_layer_sizes=50,
                              learning_rate_init=0.1, max_iter=10,
                              verbose=True))],
         verbose=True)

Here we compute the results (error) after preprocessing:

In [3]:
pip = full_pipeline_with_model.fit(X_train, y_train)

Iteration 1, loss = 8536348.73127468
Iteration 2, loss = 5904543.73683648
Iteration 3, loss = 5541758.83587787
Iteration 4, loss = 5088738.53890438
Iteration 5, loss = 4737941.30525502
Iteration 6, loss = 4493017.85475540
Iteration 7, loss = 4357043.30515104
Iteration 8, loss = 4212331.25933487
Iteration 9, loss = 4149803.98886336
Iteration 10, loss = 4057529.46521621
Iteration 11, loss = 4026307.71207706
Iteration 12, loss = 3927875.85745414
Iteration 13, loss = 3893388.96331511
Iteration 14, loss = 3829545.22747504
Iteration 15, loss = 3815743.39346727
Iteration 16, loss = 3780042.52093492
Iteration 17, loss = 3752953.87531488
Iteration 18, loss = 3756527.82443350
Iteration 19, loss = 3718780.89568251
Iteration 20, loss = 3688795.83252406
Iteration 21, loss = 3700019.86125598
Iteration 22, loss = 3661162.39016758
Iteration 23, loss = 3644846.01076794
Iteration 24, loss = 3598523.58582714
Iteration 25, loss = 3632248.51472321
Iteration 26, loss = 3576050.94293956
Iteration 27, loss = 

In [8]:
import numpy as np
predicted = model.predict(X_test)
np.sqrt(np.sum((predicted -  y_test) ** 2) / predicted.shape[0])

2720.486519244561

Here we compute the results without preprocessing:

In [ ]:
sim = baseline_full_pipeline
sim.fit(X_train, y_train)

In [4]:
import numpy as np
predicted = sim.predict(X_test)
np.sqrt(np.sum((predicted -  y_test) ** 2) / predicted.shape[0])

3769.899483967153

We can see a clear improvement.